In [1]:
!pip install transformers
!pip install torch


In [2]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pdfplumber

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased')


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
max_length = 512
chunk_size = 300

def tokenize_and_generate_summary(document):
    chunks = [document[i:i+chunk_size] for i in range(0, len(document), chunk_size)]
    summaries = []

    for chunk in chunks:
        tokens = tokenizer(chunk, return_tensors='pt', max_length=max_length, truncation=True)
        summary = model(**tokens).last_hidden_state.mean(dim=1)
        summaries.append(summary)

    return torch.cat(summaries, dim=0)

In [4]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text


In [5]:
# Replace 'your_document.pdf' with the actual path to your PDF file
document_a_path = '/Users/ananyasirandass/Downloads/01_introduction.pdf'
document_b_path = '/Users/ananyasirandass/Downloads/ML intro.pdf'

document_a_text = extract_text_from_pdf(document_a_path)
document_b_text = extract_text_from_pdf(document_b_path)

summary_a = tokenize_and_generate_summary(document_a_text)
summary_b = tokenize_and_generate_summary(document_b_text)


In [6]:
# Detach tensors and convert to NumPy arrays
summary_a_np = summary_a.squeeze().detach().numpy()
summary_b_np = summary_b.squeeze().detach().numpy()

# Reshape to a row vector
#summary_a_np = summary_a_np.reshape(1, -1)
#summary_b_np = summary_b_np.reshape(1, -1)


In [7]:
# Calculate cosine similarity
similarity_score = cosine_similarity(summary_a_np, summary_b_np)
similarity_score1 = cosine_similarity(summary_a_np, summary_a_np)
similarity_score2 = cosine_similarity(summary_b_np, summary_b_np)

# Print the result
print("Cosine Similarity Score:", similarity_score[0][0])
print("Cosine Similarity Score:", similarity_score1[0][0])
print("Cosine Similarity Score:", similarity_score2[0][0])


Cosine Similarity Score: 0.8205671
Cosine Similarity Score: 0.9999996
Cosine Similarity Score: 0.9999999
